## Projet Scala - Sales Product Data
## Ndèye Khady SEGNANE - 220798
## Djiadji DIAW - 220762
## Master I BIGL 2022/2023

## Création de la spark session

In [2]:
import $ivy.`org.apache.spark::spark-sql:2.4.5` // Or use any other 2.x version here
import $ivy.`org.apache.spark::spark-mllib:2.4.5`
import org.apache.log4j.{Level, Logger}
import org.apache.spark.sql._

Logger.getLogger("org").setLevel(Level.OFF)

val spark = {
  NotebookSparkSession.builder()
    .master("local[4]")
    .getOrCreate()
}

Getting spark JARs
Creating SparkSession


Spark UI

import $ivy.$                                   // Or use any other 2.x version here

import $ivy.$                                    

import org.apache.log4j.{Level, Logger}

import org.apache.spark.sql._


spark: SparkSession = org.apache.spark.sql.SparkSession@af236a6

## Chargement des données

In [6]:
val data = "archive/Basededata.csv"

val salesDF = spark.read
  .option("header", "true")
  .option("multiLine", "true")
  .option("inferSchema", "true")
  .option("escape", "\"")
  .csv(data)


csv at cmd5.sc:8

1 / 1

csv at cmd5.sc:8

1 / 1

csv at cmd5.sc:8

1 / 1

csv at cmd5.sc:8

1 / 1

data: String = "archive/Basededata.csv"
salesDF: DataFrame = [Order ID: string, Product: string ... 4 more fields]

In [5]:
import org.apache.spark.sql.types._

val salesProductSchema = StructType(Array(
    StructField("Order ID", IntegerType, true),
    StructField("Product", StringType, true),
    StructField("Quantity Ordered", IntegerType, true),
    StructField("Price Each", DoubleType, true),
    StructField("Order Date", StringType, true),
    StructField("Purchase Address", StringType, true)
))

import org.apache.spark.sql.types._


salesProductSchema: StructType = StructType(
  StructField("Order ID", IntegerType, true, {}),
  StructField("Product", StringType, true, {}),
  StructField("Quantity Ordered", IntegerType, true, {}),
  StructField("Price Each", DoubleType, true, {}),
  StructField("Order Date", StringType, true, {}),
  StructField("Purchase Address", StringType, true, {})
)

In [7]:
println("Le nombre de ligne est " + salesDF.count())
println("Le nombre de colonne est " + salesDF.columns.length)
salesDF.columns

count at cmd6.sc:1

1 / 1

count at cmd6.sc:1

1 / 1

count at cmd6.sc:1

1 / 1

count at cmd6.sc:1

1 / 1

Le nombre de ligne est 186316
Le nombre de colonne est 6


res6_2: Array[String] = Array(
  "Order ID",
  "Product",
  "Quantity Ordered",
  "Price Each",
  "Order Date",
  "Purchase Address"
)

#### Q1. Quelle année a été la meilleure en termes de ventes ? Combien a-t-on gagné cette année-là?

In [9]:
import org.apache.spark.sql.functions.col
import org.apache.spark.sql.functions.{to_timestamp, year}

// Convertir la colonne "Order Date" en timestamp
val spdTimestamp = salesDF.withColumn("Order Date", to_timestamp(col("Order Date"), "MM/dd/yyyy HH:mm"))

// Calculer le prix total de chaque commande
val spdWithTotalPrice = spdTimestamp.withColumn("Total Price", col("Price Each") * col("Quantity Ordered"))

// Extraire l'année à partir de la colonne "Order Date"
val spdWithYear = spdWithTotalPrice.withColumn("Year", year(col("Order Date")))

// Calculer le revenu total par année
val revenuAnnuel = spdWithYear.groupBy("Year").sum("Total Price").orderBy(col("sum(Total Price)").desc).filter(col("Year") > 1)

println("20" + revenuAnnuel.first()(0) + " a été la meilleure année en terme de vente")
println("Montant total des ventes cette année = " + revenuAnnuel.first()(1))

first at cmd8.sc:16

1 / 1

first at cmd8.sc:16

1 / 1

first at cmd8.sc:16

...

200 / 200

first at cmd8.sc:17

200 / 200

Montant total des ventes cette année = 3.379903640992757E7


import org.apache.spark.sql.functions.col

import org.apache.spark.sql.functions.{to_timestamp, year}

// Convertir la colonne "Order Date" en timestamp

spdTimestamp: DataFrame = [Order ID: string, Product: string ... 4 more fields]
spdWithTotalPrice: DataFrame = [Order ID: string, Product: string ... 5 more fields]
spdWithYear: DataFrame = [Order ID: string, Product: string ... 6 more fields]
revenuAnnuel: Dataset[Row] = [Year: int, sum(Total Price): double]

### Q2. Quel mois a été le meilleur en termes de ventes ? Combien a-t-on gagné ce mois-là ?

In [10]:
import org.apache.spark.sql.functions.month

// Extraire le mois à partir de la colonne "Order Date"
val spdWithMonth = spdWithTotalPrice.withColumn("Month", month(col("Order Date")))

// Calculer le revenu total par année
val revenuMensuel = spdWithMonth.groupBy("Month").sum("Total Price").orderBy(col("sum(Total Price)").desc).filter(col("Month") > 1)

println("Le mois " + revenuMensuel.first()(0) + " a été le meilleur mois en terme de vente")
println("Montant total des ventes ce mois-ci = " + revenuMensuel.first()(1))

first at cmd9.sc:9

1 / 1

first at cmd9.sc:9

1 / 1

first at cmd9.sc:9

...

200 / 200

first at cmd9.sc:10

200 / 200

Montant total des ventes ce mois-ci = 3715537.46000095


import org.apache.spark.sql.functions.month

// Extraire le mois à partir de la colonne "Order Date"

spdWithMonth: DataFrame = [Order ID: string, Product: string ... 6 more fields]
revenuMensuel: Dataset[Row] = [Month: int, sum(Total Price): double]

### Q3. Quelle ville a enregistré le plus grand nombre de ventes ?

In [11]:
import org.apache.spark.sql.functions.split

// Extraire la ville à partir de la colonne "Purchase Address"
val spdWithCity = spdWithTotalPrice.withColumn("City", split(col("Purchase Address"), ", ").getItem(1))

// Calculer le nombre de ventes par ville
val spdByCity = spdWithCity.groupBy("City").count().orderBy(col("count").desc)

println("La ville avec le plus grand nombre de ventes : " + spdByCity.first()(0))
println("Nombre total de ventes dans cette ville : " + spdByCity.first()(1))

first at cmd10.sc:9

1 / 1

first at cmd10.sc:9

1 / 1

first at cmd10.sc:9

...

200 / 200

first at cmd10.sc:10

200 / 200

Nombre total de ventes dans cette ville : 44732


import org.apache.spark.sql.functions.split

// Extraire la ville à partir de la colonne "Purchase Address"

spdWithCity: DataFrame = [Order ID: string, Product: string ... 6 more fields]
spdByCity: Dataset[Row] = [City: string, count: bigint]

### Q4. À quelle heure devraient-ils diffuser des publicités pour maximiser les chances que les clients achètent le produit ?

In [12]:
import org.apache.spark.sql.functions.hour

// Extraire l'heure à partir de la colonne "Order Date"
val spdWithHour = spdTimestamp.withColumn("Hour", hour(col("Order Date")))

// Calculer le nombre de vente par heure
val spdByHour = spdWithHour.groupBy("Hour").count().orderBy(col("count").desc)

println(spdByHour.first()(0) + "h est l'heure idéale pour diffuser des publicités")

first at cmd11.sc:9

1 / 1

first at cmd11.sc:9

1 / 1

first at cmd11.sc:9

200 / 200

first at cmd11.sc:9

200 / 200

19h est l'heure idéale pour diffuser des publicités


import org.apache.spark.sql.functions.hour

// Extraire l'heure à partir de la colonne "Order Date"

spdWithHour: DataFrame = [Order ID: string, Product: string ... 5 more fields]
spdByHour: Dataset[Row] = [Hour: int, count: bigint]

### Q5. Quels produits sont le plus souvent vendus ensemble ?


In [ ]:
import org.apache.spark.ml.fpm.FPGrowth
import org.apache.spark.sql.functions._

// Convertir les données en un tableau de listes de produits par transaction
val transactions = transactionsDF
  .select("product_id")
  .rdd
  .map(_.getString(0))
  .groupBy(identity)
  .map(_._2.toSeq)
  .collect()

// Définir les paramètres de l'algorithme
val minSupport = 0.2 // Support minimum (ajustez selon vos besoins)

// Créer un DataFrame de transactions pour l'algorithme FPGrowth
val transactionsDF = spark.createDataFrame(transactions.map(Tuple1.apply)).toDF("items")

// Appliquer l'algorithme FPGrowth pour extraire les combinaisons fréquentes
val fpGrowth = new FPGrowth()
  .setItemsCol("items")
  .setMinSupport(minSupport)

val model = fpGrowth.fit(transactionsDF)
val frequentItemsets = model.freqItemsets

// Afficher les combinaisons fréquentes de produits
frequentItemsets.show()


### Q6. Quel produit s'est le plus vendu ? Pourquoi pensez-vous qu'il se soit autant vendu ?

In [14]:
import org.apache.spark.sql.functions._

// Calcul du nombre de ventes par produit
val salesByProduct = salesDF
  .groupBy("product")
  .agg(count("*").as("total_sales"))
  .orderBy(desc("total_sales"))

// Affichage du produit le plus vendu
val topProduct = salesByProduct.select("product").first().getString(0)
val topSales = salesByProduct.select("total_sales").first().getLong(0)
println(s"Le produit le plus vendu est : $topProduct avec un total de ventes de $topSales")


first at cmd13.sc:10

1 / 1

first at cmd13.sc:10

1 / 1

first at cmd13.sc:10

...

200 / 200

first at cmd13.sc:11

200 / 200

Le produit le plus vendu est : USB-C Charging Cable avec un total de ventes de 21903


import org.apache.spark.sql.functions._

// Calcul du nombre de ventes par produit

salesByProduct: Dataset[Row] = [product: string, total_sales: bigint]
topProduct: String = "USB-C Charging Cable"
topSales: Long = 21903L

### Q7. Quelle est la probabilité que les prochains clients commandent un câble de chargement USB-C ?


In [21]:
import org.apache.spark.sql.functions._

// Calcul du nombre total de commandes
val totalOrders = salesDF.count()

// Calcul du nombre de commandes contenant un câble de chargement USB-C
val usbCableOrders = salesDF.filter(col("product") === "Câble de chargement USB-C")
val usbCableOrderCount = usbCableOrders.count()

// Calcul de la probabilité
val probability = usbCableOrderCount.toDouble / totalOrders.toDouble

// Affichage de la probabilité
println(s"La probabilité que les prochains clients commandent un câble de chargement USB-C est de : $probability")

count at cmd20.sc:8

1 / 1

count at cmd20.sc:8

1 / 1

count at cmd20.sc:8

...

1 / 1

count at cmd20.sc:12

1 / 1

La probabilité que les prochains clients commandent un câble de chargement USB-C est de : 0.0


import org.apache.spark.sql.functions._

// Chargement des données de commandes à partir d'un DataFrame
//val ordersDF = spark.read.csv("archive/CommandeDF.csv")
//.toDF("order_id", "product")

// Calcul du nombre total de commandes

totalOrders: Long = 186316L
usbCableOrders: Dataset[Row] = [Order ID: string, Product: string ... 4 more fields]
usbCableOrderCount: Long = 0L
probability: Double = 0.0

### Q8. Quelle est la probabilité que les prochains clients commandent un iPhone ?

In [23]:
import org.apache.spark.sql.functions._

// Calcul du nombre total de commandes
val totalOrders = salesDF.count()

// Calcul du nombre de commandes contenant un iPhone
val iphoneOrders = salesDF.filter(col("product") === "iPhone")
val iphoneOrderCount = iphoneOrders.count()

// Calcul de la probabilité
val probability = (iphoneOrderCount.toDouble / totalOrders.toDouble)*100

// Affichage de la probabilité
println(s"La probabilité que les prochains clients commandent un iPhone est de : $probability")



count at cmd22.sc:8

1 / 1

count at cmd22.sc:8

1 / 1

count at cmd22.sc:8

...

1 / 1

count at cmd22.sc:12

1 / 1

La probabilité que les prochains clients commandent un iPhone est de : 3.672255737564138


import org.apache.spark.sql.functions._

// Chargement des données de commandes à partir d'un DataFrame
//val ordersDF = spark.read.csv("chemin/vers/votre/fichier/commandes.csv")
//  .toDF("order_id", "product")

// Calcul du nombre total de commandes

totalOrders: Long = 186316L
iphoneOrders: Dataset[Row] = [Order ID: string, Product: string ... 4 more fields]
iphoneOrderCount: Long = 6842L
probability: Double = 3.672255737564138

### 9. Quelle est la probabilité que les prochains clients commandent un téléphone Google ?

In [28]:
import org.apache.spark.sql.functions._

// Calcul du nombre total de commandes
val totalOrders = salesDF.count()

// Calcul du nombre de commandes contenant un téléphone Google
val googlePhoneOrders = salesDF.filter(col("product").contains("Google"))
val googlePhoneOrderCount = googlePhoneOrders.count()

// Calcul de la probabilité
val probability = (googlePhoneOrderCount.toDouble / totalOrders.toDouble)*100

// Affichage de la probabilité
println(s"La probabilité que les prochains clients commandent un téléphone Google est de : $probability")

count at cmd27.sc:8

1 / 1

count at cmd27.sc:8

1 / 1

count at cmd27.sc:8

...

1 / 1

count at cmd27.sc:12

1 / 1

La probabilité que les prochains clients commandent un téléphone Google est de : 2.965392129500419


import org.apache.spark.sql.functions._

// Chargement des données de commandes à partir d'un DataFrame
//val ordersDF = spark.read.csv("archive/CommandeDF.csv")
//.toDF("Order id", "product")

// Calcul du nombre total de commandes

totalOrders: Long = 186316L
googlePhoneOrders: Dataset[Row] = [Order ID: string, Product: string ... 4 more fields]
googlePhoneOrderCount: Long = 5525L
probability: Double = 2.965392129500419

### 10. Quelle est la probabilité que d'autres personnes commandent des écouteurs filaires ?

In [30]:
import org.apache.spark.sql.functions._

// Calcul du nombre total de commandes
val totalOrders = salesDF.count()

// Calcul du nombre de commandes contenant des écouteurs filaires
val wiredHeadphonesOrders = salesDF.filter(col("product").contains("écouteurs filaires"))
val wiredHeadphonesOrderCount = wiredHeadphonesOrders.count()

// Calcul de la probabilité
val probability = wiredHeadphonesOrderCount.toDouble / totalOrders.toDouble

// Affichage de la probabilité
println(s"La probabilité que d'autres personnes commandent des écouteurs filaires est de : $probability")

count at cmd29.sc:8

1 / 1

count at cmd29.sc:8

1 / 1

count at cmd29.sc:8

...

1 / 1

count at cmd29.sc:12

1 / 1

La probabilité que d'autres personnes commandent des écouteurs filaires est de : 0.0


import org.apache.spark.sql.functions._

// Chargement des données de commandes à partir d'un DataFrame
//val ordersDF = spark.read.csv("chemin/vers/votre/fichier/commandes.csv")
 // .toDF("order_id", "product")

// Calcul du nombre total de commandes

totalOrders: Long = 186316L
wiredHeadphonesOrders: Dataset[Row] = [Order ID: string, Product: string ... 4 more fields]
wiredHeadphonesOrderCount: Long = 0L
probability: Double = 0.0

### Proposer un cas d'utilisation concret de Machine Learning en utilisant cet ensemble de données de ventes. Vous devrez identifier un problème spécifique et proposer un modèle prédictif ou analytique qui peut être utilisé pour résoudre le problème.

### L'objectif est de développer un modèle prédictif qui est capable de prédire les ventes de produits pour une période spécifique, pour le mois suivant. Cela permettra à l'entreprise de planifier sa production, d'ajuster ses stocks et de prendre des décisions commerciales éclairées.

### Séparation des données d'entrainement et des données test

In [32]:
val Array(trainDF, testDF) = salesDF.randomSplit(Array(.8, .2), seed=28)
println(f"Il y'a ${trainDF.cache().count()} lignes dans le training set, et ${testDF.cache().count()} dans le test set")

count at cmd31.sc:2

1 / 1

count at cmd31.sc:2

1 / 1

count at cmd31.sc:2

...

1 / 1

count at cmd31.sc:2

1 / 1

Il y'a 148906 lignes dans le training set, et 37410 dans le test set


trainDF: Dataset[Row] = [Order ID: string, Product: string ... 4 more fields]
testDF: Dataset[Row] = [Order ID: string, Product: string ... 4 more fields]

In [35]:
salesDF.printSchema

root
 |-- Order ID: string (nullable = true)
 |-- Product: string (nullable = true)
 |-- Quantity Ordered: string (nullable = true)
 |-- Price Each: string (nullable = true)
 |-- Order Date: string (nullable = true)
 |-- Purchase Address: string (nullable = true)



In [36]:
salesDF.select("Price Each").printSchema

root
 |-- Price Each: string (nullable = true)



In [34]:
import org.apache.spark.ml.feature.VectorAssembler 

val vecAssembler = new VectorAssembler()
  .setInputCols(Array("Product")) 
  .setOutputCol("features")

val vecTrainDF = vecAssembler.transform(trainDF)

vecTrainDF
  .select("Product", "features", "Price Each")
  .show(10)

: 